In [1]:
from selenium.common.exceptions import TimeoutException
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from appium.options.android import UiAutomator2Options
from appium import webdriver
import pandas as pd
from datetime import datetime
import time

In [ ]:
# ===================== 설정 =====================
APP_PACKAGE = "kr.or.keis.mo"

LOGIN_ID = "아이디 입력" # 아이디 입력
LOGIN_PW = "비밀번호 입력" # 비밀번호 입력

APPIUM_SERVER_URL = "http://127.0.0.1:4723"
DEVICE_NAME = "Android"

# 반복 횟수 (예: 10번 로그인)
REPEAT_COUNT = 10

In [3]:
# 아이디/비밀번호(HRD 출결용) 버튼 클릭
def tap_idpw_menu(driver, wait):
    """'아이디/비밀번호(HRD 출결용)' 버튼 클릭"""
    print("📲 [3단계] '아이디/비밀번호(HRD 출결용)' 버튼 클릭")

    # 화면 아래쪽 보이도록 한 번 스와이프
    try:
        driver.swipe(22, 942, 22, 650, 500)  # (startX, startY, endX, endY)
        time.sleep(1)
        print("   ✅ 스와이프 수행 완료")
    except Exception as e:
        print(f"   ℹ️ 스와이프 중 오류(무시): {e}")

    # UiSelector(resourceId) 로 버튼 클릭
    try:
        btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.ANDROID_UIAUTOMATOR,
                 'new UiSelector().resourceId("btn_idpopup")')
            )
        )
        btn.click()
        print("   ✅ '아이디/비밀번호(HRD 출결용)' 버튼 클릭 성공")
    except TimeoutException as e:
        print("   ❌ '아이디/비밀번호(HRD 출결용)' 버튼을 찾지 못했습니다.")
        raise RuntimeError("ID/PW(HRD 출결용) 메뉴를 찾지 못했습니다.") from e

In [4]:
# 아이디, 비밀번호 입력

def fill_login_form(driver, wait):
    """로그인 화면에서 ID/PW 입력"""
    print("⌨️ [4단계] 아이디 / 비밀번호 자동 입력")

    def _two_edittexts(d):
        els = d.find_elements(AppiumBy.CLASS_NAME, "android.widget.EditText")
        return els if len(els) >= 2 else False

    try:
        inputs = wait.until(_two_edittexts)
        id_input, pw_input = inputs[0], inputs[1]

        id_input.clear()
        id_input.send_keys(LOGIN_ID)
        print("   ✅ ID 입력 완료")

        pw_input.clear()
        pw_input.send_keys(LOGIN_PW)
        print("   ✅ PW 입력 완료")
    except Exception as e:
        print("   ❌ 로그인 입력창을 찾지 못했습니다.")
        raise RuntimeError("로그인 ID/PW 입력창을 찾지 못했습니다.") from e

In [5]:
# 로그인, 로그아웃 상태 판단

def open_login_section(driver, wait):
    """
    '로그인을 해 주세요' 영역을 눌러 로그인 수단 선택 화면으로 이동.
    메뉴가 닫혀 있으면 먼저 '전체메뉴' 버튼을 클릭해서 연다.
    """
    print("📲 [1단계] '로그인을 해 주세요'로 이동")

    try:
        # 이미 전체메뉴가 열려 있는 경우 바로 찾기
        login_please = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable(
                (AppiumBy.ACCESSIBILITY_ID, "로그인을 해 주세요")
            )
        )
        print("   ✅ 메뉴가 이미 열려 있음 → 바로 '로그인을 해 주세요' 클릭")
    except TimeoutException:
        # 안 보이면 '전체메뉴' 먼저 클릭
        print("   ℹ️ '로그인을 해 주세요'가 안 보여서 '전체메뉴' 버튼부터 클릭")
        all_menu_btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.XPATH, "//android.widget.Button[@text='전체메뉴']")
            )
        )
        all_menu_btn.click()

        login_please = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.ACCESSIBILITY_ID, "로그인을 해 주세요")
            )
        )

    login_please.click()
    print("   ✅ '로그인을 해 주세요' 클릭 완료")

    print("⏳ [1-1단계] 화면 전환 안정화를 위해 3초 대기...")
    time.sleep(3)

In [6]:
# 로그아웃

def logout_from_all_menu(driver, wait):
    """
    전체메뉴에서 '로그아웃' 버튼을 눌러 로그아웃.
    (2번째 로그인 시도부터 사용)
    """
    print("🚪 [사전작업] 전체메뉴에서 로그아웃 시도")

    # 1) '전체메뉴' 버튼 클릭 (이미 열려 있으면 예외 나도 무시)
    try:
        all_menu_btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.XPATH, "//android.widget.Button[@text='전체메뉴']")
            )
        )
        all_menu_btn.click()
        print("   ✅ '전체메뉴' 버튼 클릭")
    except TimeoutException:
        print("   ℹ️ '전체메뉴' 버튼을 찾지 못했습니다. (이미 열려 있을 수도 있음)")

    # 2) '로그아웃' 텍스트뷰 클릭
    try:
        logout_btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.ANDROID_UIAUTOMATOR,
                 'new UiSelector().text("로그아웃")')
            )
        )
        logout_btn.click()
        print("   ✅ '로그아웃' 버튼 클릭 성공")
    except TimeoutException:
        print("   ℹ️ '로그아웃' 버튼을 찾지 못했습니다. 이미 로그아웃 상태일 수 있습니다.")
        return

    # 3) 로그아웃 확인 팝업 (있으면 처리, 없으면 패스)
    try:
        confirm_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (AppiumBy.ID, "android:id/button1")
            )
        )
        confirm_btn.click()
        print("   ✅ 로그아웃 확인 팝업 처리 완료")
    except TimeoutException:
        print("   ℹ️ 로그아웃 확인 팝업이 없거나 자동으로 닫혔습니다.")

    time.sleep(2)

In [7]:
# 최초 로그인 시행

def perform_login_once(driver, wait):
    """
    로그인 시나리오 1회 수행:
    '로그인을 해 주세요' → '아이디/비밀번호(HRD 출결용)' → ID/PW 입력 →
    로그인 버튼 클릭 후 팝업 메시지가 뜰 때까지 시간 측정.

    return: (elapsed_seconds, popup_first_line)
    """
    print("🚀 [로그인 시나리오] 시작")

    # [1단계] '로그인을 해 주세요' 진입
    open_login_section(driver, wait)

    # [3단계] 아이디/비밀번호(HRD 출결용)
    tap_idpw_menu(driver, wait)

    # [4단계] ID/PW 입력
    fill_login_form(driver, wait)

    # [5단계] 로그인 버튼 클릭 + 응답 시간 측정
    print("⏱️ [5단계] 로그인 버튼 클릭 후 응답 속도 측정 시작")

    # 키보드 숨기기 (가려져 있을 수 있으니)
    try:
        driver.hide_keyboard()
        time.sleep(0.5)
        print("   📱 소프트 키보드 숨김")
    except Exception:
        pass

    # 시간을 재는 구간: 버튼 클릭 ~ 팝업 등장
    login_btn = wait.until(
        EC.element_to_be_clickable(
            (AppiumBy.ANDROID_UIAUTOMATOR,
             'new UiSelector().resourceId("btnIndvIdLogin")')
        )
    )

    start_time = time.time()
    login_btn.click()

    popup = wait.until(
        EC.presence_of_element_located((AppiumBy.ID, "android:id/message"))
    )
    end_time = time.time()

    elapsed = end_time - start_time
    popup_first_line = popup.text.splitlines()[0] if popup.text else ""

    print("\n🎉 로그인 응답 수신!")
    print(f"🚀 로그인 반응 속도: {elapsed:.4f} 초")
    print(f"📄 팝업 내용: {popup_first_line}")

    # 팝업 확인 버튼
    try:
        driver.find_element(AppiumBy.ID, "android:id/button1").click()
        print("   ✅ 팝업 확인 버튼 클릭")
    except Exception:
        print("   ℹ️ 팝업 확인 버튼을 찾지 못했습니다.")

    return elapsed, popup_first_line

In [8]:
# 메인 테스트
def test_login_security_safe(repeat_count=REPEAT_COUNT):

    options = UiAutomator2Options()
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_activity = ".MainActivity"
    options.automation_name = "UiAutomator2"
    options.new_command_timeout = 300
    options.no_reset = True

    print("--- 로그인 성능 테스트 (반복) ---")
    driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
    wait = WebDriverWait(driver, 20)

    results = []  # 각 회차 결과를 저장
    try:
        for i in range(1, repeat_count + 1):
            print("\n" + "=" * 60)
            print(f"🔁 로그인 시도 {i}/{repeat_count}")
            print("=" * 60)

            # 2번째 시도부터는 로그아웃 먼저 수행
            if i >= 2:
                logout_from_all_menu(driver, wait)

            # 로그인 1회 수행 + 시간 측정
            elapsed, popup_first_line = perform_login_once(driver, wait)

            results.append({
                "회차": i,
                "측정시각": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "로그인반응속도(초)": round(elapsed, 4),
                "팝업메시지": popup_first_line,
            })

            # 회차 사이 약간의 대기 (너무 빠르게 반복 방지)
            time.sleep(2)

        print("\n✅ 모든 반복 로그인 테스트 완료")

    except Exception as e:
        print("\n❌ 테스트 도중 실패")
        print(f"에러 내용: {e}")

    finally:
        driver.quit()
        print("\n✅ 드라이버 종료 완료")

    # ===================== CSV + 통계 저장 =====================
    print("\n" + "=" * 50)
    print("💾 로그인 성능 결과 CSV 저장 중...")

    if results:
        df = pd.DataFrame(results)

        # ---- 통계 계산 (평균, 최소, 최대, 표준편차) ----
        speeds = df["로그인반응속도(초)"]

        mean_val = speeds.mean()
        min_val = speeds.min()
        max_val = speeds.max()

        # 시도가 2번 이상이면 표본 표준편차, 1번이면 0으로 처리
        if len(speeds) >= 2:
            std_val = speeds.std(ddof=1)
        else:
            std_val = 0.0

        # 반올림
        mean_val = round(mean_val, 4)
        min_val = round(min_val, 4)
        max_val = round(max_val, 4)
        std_val = round(std_val, 4)

        print("\n📊 통계 요약")
        print(f"   평균: {mean_val} 초")
        print(f"   최소: {min_val} 초")
        print(f"   최대: {max_val} 초")
        print(f"   표준편차: {std_val} 초")

        # ---- 요약 행을 DataFrame 맨 아래에 추가 ----
        summary_row = {
            "회차": "요약",
            "측정시각": "",
            "로그인반응속도(초)": "",
            #"팝업메시지": "",
            "평균(초)": mean_val,
            "최소(초)": min_val,
            "최대(초)": max_val,
            "표준편차(초)": std_val,
        }

        df = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)

        # ---- CSV 저장 ----
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"login_perf_result_{repeat_count}runs_{timestamp}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")

        print(f"\n✅ CSV 저장 완료! 파일명: {file_name}")
        print(df)
    else:
        print("ℹ️ 저장할 측정 결과가 없어 CSV는 생성하지 않습니다.")

    print("\n✅ 로그인 반복 테스트 및 CSV 저장까지 모두 완료되었습니다.")

In [9]:
# 테스트 실행

if __name__ == "__main__":
    # 기본값 10번 반복; 필요하면 REPEAT_COUNT나 인자 바꿔서 실행
    test_login_security_safe()

--- 로그인 성능 테스트 (반복) ---

🔁 로그인 시도 1/10
🚀 [로그인 시나리오] 시작
📲 [1단계] '로그인을 해 주세요'로 이동
   ℹ️ '로그인을 해 주세요'가 안 보여서 '전체메뉴' 버튼부터 클릭
   ✅ '로그인을 해 주세요' 클릭 완료
⏳ [1-1단계] 화면 전환 안정화를 위해 3초 대기...
📲 [3단계] '아이디/비밀번호(HRD 출결용)' 버튼 클릭
   ✅ 스와이프 수행 완료
   ✅ '아이디/비밀번호(HRD 출결용)' 버튼 클릭 성공
⌨️ [4단계] 아이디 / 비밀번호 자동 입력
   ✅ ID 입력 완료
   ✅ PW 입력 완료
⏱️ [5단계] 로그인 버튼 클릭 후 응답 속도 측정 시작
   📱 소프트 키보드 숨김

🎉 로그인 응답 수신!
🚀 로그인 반응 속도: 1.5210 초
📄 팝업 내용: 로그인 성공하였습니다.
   ✅ 팝업 확인 버튼 클릭

🔁 로그인 시도 2/10
🚪 [사전작업] 전체메뉴에서 로그아웃 시도
   ✅ '전체메뉴' 버튼 클릭
   ✅ '로그아웃' 버튼 클릭 성공
   ℹ️ 로그아웃 확인 팝업이 없거나 자동으로 닫혔습니다.
🚀 [로그인 시나리오] 시작
📲 [1단계] '로그인을 해 주세요'로 이동
   ℹ️ '로그인을 해 주세요'가 안 보여서 '전체메뉴' 버튼부터 클릭
   ✅ '로그인을 해 주세요' 클릭 완료
⏳ [1-1단계] 화면 전환 안정화를 위해 3초 대기...
📲 [3단계] '아이디/비밀번호(HRD 출결용)' 버튼 클릭
   ✅ 스와이프 수행 완료
   ✅ '아이디/비밀번호(HRD 출결용)' 버튼 클릭 성공
⌨️ [4단계] 아이디 / 비밀번호 자동 입력
   ✅ ID 입력 완료
   ✅ PW 입력 완료
⏱️ [5단계] 로그인 버튼 클릭 후 응답 속도 측정 시작
   📱 소프트 키보드 숨김

🎉 로그인 응답 수신!
🚀 로그인 반응 속도: 1.3218 초
📄 팝업 내용: 로그인 성공하였습니다.
   ✅ 팝업 확인 버튼 클릭

🔁 로그인 시도 3/10
🚪 [사전작업] 전체메뉴에서 로그아웃 시도
   ✅ '전체